<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.6 MB/s eta 0:00:00


In [2]:
!pip install  kili

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.0/293.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.3/213.3 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.9 MB/s eta 0:00:00
  Created wheel for cuid: filename=cuid-0.4-py2.py3-none-any.whl size=4714 sha256=1debddd1b53ca3249f60ea07f13bf9775d047dee708bc6d93febd045e11f3d07
  Stored in directory: /root/.cache/pip/wheels/52/0a/dc/a0e28c435d5a74d9eef3d7c3cd147b96cb21e71e5ec7dcfdbe
Successfully built cuid


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [11]:
import pickle

with open(f"{base_path}conversations_with_reasoning.pkl", 'rb') as file:
    conversations = pickle.load(file)

print("Len loaded data:", len(conversations))

Len loaded data: 32990


In [12]:
for id in conversations:
  print(conversations[id]['comments'][0])
  print(conversations[id]['pairs'][0])
  break

{'id': 'xyt38s', 'body': 'MLB Sat Info\n\nPlay\xa0OnAll underdogs with a money line of +100 or higher\xa0(SEATTLE)team with a terrible SLG (&lt;=.400) against a very good starting pitcher (WHIP&lt;=1.300) -AL, with a very good bullpen whose WHIP is 1.250 or better on the season217-224\xa0over the last 5 seasons.49.2%\xa0(89.1\xa0units)', 'author': None, 'reply_to': None, 'conversation_id': 'xyt38s', 'politness_score': 0.09523809523809523, 'readability_score': -0.5, 'has_reasoning': False}
{'comment': 'MLB Sat Info\n\nPlay\xa0OnAll underdogs with a money line of +100 or higher\xa0(SEATTLE)team with a terrible SLG (&lt;=.400) against a very good starting pitcher (WHIP&lt;=1.300) -AL, with a very good bullpen whose WHIP is 1.250 or better on the season217-224\xa0over the last 5 seasons.49.2%\xa0(89.1\xa0units)', 'reply': 'Play\xa0OnAll underdogs with a money line of +100 or higher\xa0(TAMPA BAY)team with a terrible SLG (<=.400) against a very good starting pitcher (WHIP<=1.300) -AL, with 

In [16]:
import random

def get_random_pairs(conversations, sample_size):
  random_pairs = []

  random_sample = random.sample(conversations.items(), sample_size)
  for sample_conversation in random_sample:
      random_pair = random.choice(sample_conversation[1]['pairs'])
      for pair in sample_conversation[1]['pairs']:
        if pair['comment_id'] == random_pair['comment_id'] and pair['reply_id'] != random_pair['reply_id']:
          other_response = pair
          random_pairs.append(random_pair)
          random_pairs.append(pair)
          break
  return random_pairs

In [28]:
len(get_random_pairs(conversations, 100))

<ipython-input-16-739721b51061>:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_sample = random.sample(conversations.items(), sample_size)


110

In [10]:
import os
import json
from kili.client import Kili

def create_new_kili_projcet_for_pairs(random_pairs):
  kili = Kili(api_key='6acd489f-9746-4ac4-9650-54e2d9c9faba')
  interface = {
      "jobs": {
          "JOB_0": {
              "mlTask": "CLASSIFICATION",
              "required": 1,
              "isChild": False,
              "content": {
                  "categories": {"IsConversation": {"name": "has a common purpose or set of purposes, or at least a mutually accepted direction"}, "NotConversation": {"name": "has no prupose or a direction"}},
                  "input": "radio",
              },
          },
          "JOB_1": {
              "mlTask": "CLASSIFICATION",
              "required": 1,
              "isChild": False,
              "content": {
                  "categories": {"Related": {"name": "is relevent"}, "UnRelated": {"name": "is not relevent"}},
                  "input": "radio",
              },
          },
          "JOB_2": {
              "mlTask": "CLASSIFICATION",
              "required": 1,
              "isChild": False,
              "content": {
                  "categories": {"HasQuality": {"name": "has quality: does not say which it lack adequate evidence or it believes to be false."}, "NonQuality": {"name": "no quality: says which it lacks evidence or it believes is false."}},
                  "input": "radio",
              },
          },
          "JOB_3": {
              "mlTask": "CLASSIFICATION",
              "required": 1,
              "isChild": False,
              "content": {
                  "categories": {"HasQuantity": {"name": "has quantity: is as informative as is required"}, "NoQuantity": {"name": "no  quantity: is more or less informative than is required"}},
                  "input": "radio",
              },
          },
          "JOB_4": {
              "mlTask": "CLASSIFICATION",
              "required": 1,
              "isChild": False,
              "content": {
                  "categories": {"HasManner": {"name": "has manner: avoids vagueness, avoids ambiguty, do not play with words, is orderly"}, "NoManner": {"name": "no manner: is vague or ambiguous or plays with words or is not orderly"}},
                  "input": "radio",
              },
          },
          "JOB_5": {
              "mlTask": "CLASSIFICATION",
              "required": 1,
              "isChild": False,
              "content": {
                  "categories": {"IsPolite": {"name": "is polite"}, "NotPolite": {"name": "is not polite"}},
                  "input": "radio",
              },
          }
      }
  }

  project = kili.create_project(
      title="Conversation and Maxims",
      description="Project Description",
      input_type="TEXT",
      json_interface=interface,
  )
  project_id = project['id']
  print("Project ID: ", project_id)
  assets = kili.append_many_to_dataset(
      project_id=project_id,
      content_array = [('comment: ' + d['comment'] + '\n' + 'reply: ' + d['reply']) for d in random_pairs[:100]],
      external_id_array = [d['global_id'] for d in random_pairs[:100]]
  )
  # Project ID:  clp5pxaet1wdn086r7y3x0ngo